### Carregando dependências

In [38]:
import pandas as pd
pd.options.mode.chained_assignment = None
import pickle
import json
import requests
import numpy as np
from app.utils.preprocessing import data_preprocessing

### Caminhos para as bases utilizadas como exemplo para o POST no endpoint de aderência:

In [39]:
# arquivos locais do repositório base
# os caminhos são relativos à API 
path_train = '../../datasets/credit_01/train.gz'
path_oot   = '../../datasets/credit_01/oot.gz'

##### O corpo da requisição é montado de acordo com a documentação. 

In [40]:
req_json = {'req-dataset': path_oot}
home = 'http://localhost:8001'
ad_endpoint = '/aderencia'
headers = {'Content-Type': 'application/json', 'accept': 'application/json'}

# POST feito no endpoint /aderencia
res_oot = requests.post(home + ad_endpoint, json=req_json, headers=headers)

##### A resposta da requisição é obtida.

Nesse caso, percebe-se que o <em><strong>p-value</strong></em> está na ordem de $10^{-12}$, o que, de acordo com o teste KS, indica que as duas amostras não vêm da mesma distribuição considerando um alfa = 0.05.

In [41]:
res_parsed = json.loads(res_oot.text)
print(json.dumps(res_parsed, indent=4))

{
    "KStest-result": {
        "statistic": 0.020915414151451373,
        "p-value": 4.016241889587134e-12
    }
}


##### Agora, para a base de treino, o corpo da requisição também é montado de acordo com a documentação.

In [42]:
req_json = {'req-dataset': path_train}

# POST feito no endpoint /aderencia
res_train = requests.post(home + ad_endpoint, json=req_json, headers=headers)

##### As métricas são obtidas.

Para a base de treino, percebe-se um <em><strong>p-value</strong></em> da ordem de $10^{-1}$, o que, de acordo com o teste KS, indica que as duas amostras são da mesma distribuição de dados considerando um alfa = 0.05.

In [43]:
res_parsed = json.loads(res_train.text)
print(json.dumps(res_parsed, indent=4))

{
    "KStest-result": {
        "statistic": 0.002759858953621075,
        "p-value": 0.9605978662359891
    }
}


### Leitura do <em>batch_records.json</em> para envio como JSON para o POST no endpoint de performance:

In [44]:
records_file = open('batch_records.json')
records_json = json.load(records_file)

##### Tem tipo List[dict] assim como esperado pelo callback do endpoint

In [45]:
type(records_json)

list

In [46]:
type(records_json[0])

dict

##### POST é feito para se obter volumetria dos dados e a performance do <strong><em>default model</em></strong>.

In [47]:
home = 'http://localhost:8001'
default_endpoint = '/performance/default'
headers = {'Content-Type': 'application/json', 'accept': 'application/json'}

# POST feito no endpoint /performance/{model_from}
res_default = requests.post(home + default_endpoint, json=records_json, headers=headers)

A resposta é obtida.

In [48]:
res_default_parsed = json.loads(res_default.text)
print(json.dumps(res_default_parsed, indent=4))

{
    "volumetria": {
        "JUL": 74,
        "AGO": 72,
        "MAI": 67,
        "JUN": 63,
        "MAR": 62,
        "JAN": 58,
        "FEV": 55,
        "ABR": 49
    },
    "roc_score": 0.5751748251748252
}


##### POST feito para se obter métricas de performance agora do <strong><em>enhanced model</em></strong>.

In [49]:
enhanced_endpoint = '/performance/enhanced'

# POST feito no endpoint /performance/{model_from}
res_enhanced = requests.post(home + enhanced_endpoint, json=records_json, headers=headers)

Resposta obtida.

In [50]:
res_enhanced_parsed = json.loads(res_enhanced.text)
print(json.dumps(res_enhanced_parsed, indent=4))

{
    "volumetria": {
        "JUL": 74,
        "AGO": 72,
        "MAI": 67,
        "JUN": 63,
        "MAR": 62,
        "JAN": 58,
        "FEV": 55,
        "ABR": 49
    },
    "roc_score": 0.6108318764568764
}


#### Considerando como métrica de performance do modelo em questão como sendo a área sob a curva ROC, o modelo enhanced tem um ganho percentual no score de:

In [51]:
ganho_percentual = res_enhanced_parsed['roc_score'] - res_default_parsed['roc_score']

In [54]:
print(str(round(ganho_percentual*100, 2)) + '%')

3.57%
